In [37]:
import pandas as pd
import numpy as np

In [38]:
def get_quality(df_num):
    df_med_dev = df_num.transform(lambda x: abs(x - x.median())).transform(lambda x: (x*0) + x.median() * 1.4826)
    df_mscore = df_num.transform(lambda x: x - x.median()) / df_med_dev
    df_quality = df_mscore.mean(axis=1).transform(lambda x: 1 / (1 + np.exp(-x))).to_frame('quality')
    return df_quality

In [39]:
df_csv = pd.read_csv('C:/Users/Haniff/Documents/GeneralProgramming/Diversified Projects/d3farmer/fsharp/Runs.csv')
dat_len = len(df_csv) - 100

In [40]:
pick_len = int(100 + dat_len/2)

In [41]:
df_in = df_csv.tail(pick_len)

In [42]:
df = df_in[['LevelName', 'LegendariesPH', 'ShardsPH']]

In [43]:
df_num = df_in[['LegendariesPH', 'ShardsPH']]

In [44]:
df_quality = get_quality(df_num)

In [45]:
df.tail(10)

,LevelName,LegendariesPH,ShardsPH
151,T2,15.6,937
152,T9,27.4,1287
153,T9,30.2,1287
154,T9,14.7,1052
155,T11,10.2,0
156,T9,12.7,826
157,T7,47.6,953
158,T7,0.0,1198
159,Master,14.4,620
160,T9,27.8,966


In [46]:
df = df.join(df_quality).sort_values('quality', ascending=False)

In [47]:
df.head(10)

,LevelName,LegendariesPH,ShardsPH,quality
139,T9,10.6,4524,0.987466
65,T4,23.8,2163,0.900871
141,T9,37.3,1653,0.900060
123,T8,12.5,2540,0.896311
116,T8,49.6,1017,0.879929
157,T7,47.6,953,0.861991
113,T8,39.9,1022,0.829320
153,T9,30.2,1287,0.813080
128,T7,37.1,950,0.798568
152,T9,27.4,1287,0.794107


In [48]:
dfgb = df.groupby(['LevelName'])

In [49]:
dfgb = dfgb['quality'].agg(['count','mean', 'var']).reset_index()

In [50]:
df = df.merge(dfgb)

In [51]:
dfgb['total'] = dfgb['count'].sum()

In [52]:
import math
def get_ucb(row):
    div1 = 2 * math.log(row['total']) / row['count']
    min1 = min(0.25, row['var'] + div1)
    div2 = math.log(row['total']) / row['count']
    ucb = math.sqrt(div2 * min1)
    return ucb + row['mean']

In [53]:
dfgb['Next Action'] = dfgb.apply(get_ucb, axis=1)

In [54]:
dfgb.sort_values('Next Action', ascending=False)

,LevelName,count,mean,var,total,Next Action
14,T9,19,0.649222,0.035762,130,0.902297
12,T7,14,0.607144,0.030567,130,0.901967
3,T10,3,0.258012,0.006388,130,0.894901
1,Master,6,0.443803,0.006924,130,0.894152
7,T2,6,0.442200,0.013670,130,0.892549
8,T3,5,0.387522,0.009548,130,0.880854
4,T11,3,0.239284,0.002255,130,0.876173
11,T6,12,0.546258,0.017804,130,0.864702
10,T5,7,0.443541,0.010245,130,0.860483
13,T8,23,0.621617,0.029394,130,0.851635
